In [ ]:
import numpy as np
import pandas as pd
import random
import time
from multiprocessing import Pool

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from math import sqrt
import xgboost as xgb

# For data visualizatoin
import matplotlib.pyplot as plt
from pandas.tools.plotting import autocorrelation_plot
from bokeh.io import output_notebook, show
from bokeh.models import Title
from bokeh.plotting import figure, output_file, show

import seaborn as sns
%matplotlib inline

from datetime import datetime, timedelta, date
from tqdm import tqdm            #for .py version
# from tqdm import tqdm_notebook as tqdm     # for .ipynb version

In [ ]:
# The dict 'params' consists of all the parameters used in the simulation software for ease of alteration
params = {
#         Set the regression model related parameters
          'train_start_dt':'2013-01',
          'train_stop_dt':'2013-12',
          'y_variable': 'trial_d',
          'X_variables':['trial_n', 'low', 'normal', 'high', 'WIND_DIRECTION', 
                         'WIND_SPEED', 'VISIBILITY', 'MSL_PRESSURE',
                         'AIR_TEMPERATURE', 'DEWPOINT', 'WETB_TEMP', 
                         'STN_PRES', 'WMO_HR_SUN_DUR', 'hour', 'day'],
    
#         Set XGBoost regression parameters (for consumption model)
          'n_estimators': 2000,
          'early_stopping_rounds': 50,  #stop if 50 consequent rounds without decrease of error
          'verbose': False,             # Change verbose to True if you want to see it train
          'nthread': 4,
    
#         Set simulator parameters to default values
          'season': 3,
          'day_of_week': 3,
          'special_event': 0,
          'tariff_policy':[],
    
#         Set Occupant behaviour dynamics
          'active_users': .5,     # Set the % of users who are willing to engage in the experiments
          'avail_users': .5,       # Set the % of users who will be available to participate in specific experiment
          'user_latency': 0,         # Set the values which correspond to real life participation delay for users 
          'init_samples': 50,      # Set the initial random samples to be chosen
          'frac_users_exp':1      # Fraction of users selected for a particular trial
          'total_experiments':100
    
         }

## Structure of the algorithm

The following code is divided in two classes namely, 

- ConsumptionModel
- Simulator

Consumption model primarily consists of the functions related to the basic machine learning model which predicts the dynamic user behaviour

In [23]:
class Simulator:
    
    def __init__(self, df, df_weather, params):
        self.params = params
        self.df = df
        self.df_weather = df_weather
        active_users = int(len(df.columns)*self.params["active_users"])   # get no. of active users from input percentage
        self.active_users = random.sample(list(df.columns), active_users)
        self.noisy_tariff = {}
        self.spring = [3, 4, 5]
        self.summer = [6, 7, 8]
        self.autumn = [9, 10, 11]
        self.winter = [1, 2, 12]


    def select_day(self):
#         Get user ids of participating users
        self.fuzzy_participation()
        
#         Select the season
        if self.params["season"] == -1:
            month = random.randrange(1,12)
        elif self.params["season"] == 0:
            month = random.choice(self.spring)
        elif self.params["season"] == 1:
            month = random.choice(self.summer)
        elif self.params["season"] == 2:
            month = random.choice(self.autumn)
        elif self.params["season"] == 3:
            month = random.choice(self.winter)
            
#         Select the day of week
        if self.params["day_of_week"] == -1:
#             Select random day
            dow = random.randrange(0,7)
        else:
            dow = self.params["day_of_week"] 
            
#         Select the random day from the entries which satisfy above conditions
        shortlist = self.df.loc[(self.df.index.month == month) & (self.df.index.dayofweek == dow), :].index
        day = random.choice(shortlist.day.values)
        year = random.choice(shortlist.year.values)
        timestamp = str(year)+"-"+str(month)+"-"+str(day)
        self.sample = self.df.loc[timestamp,self.avail_users]
        
        
        
    def random_day(self):
#         Get user ids of participating users
        self.fuzzy_participation()
    
#         Sample a random day timestamp
        shortlist = self.df.sample(axis = 0).index
        day = random.choice(shortlist.day.values)
        month = random.choice(shortlist.month.values)
        year = random.choice(shortlist.year.values)
        timestamp = str(year)+"-"+str(month)+"-"+str(day)
        self.sample = self.df.loc[timestamp,self.avail_users]
        
        
    
    def fuzzy_participation(self):
        avail_users = int(len(self.active_users)*self.params["avail_users"])
        self.avail_users = random.sample(self.active_users, avail_users)
    
    
    def auto_noise_addition(self, levels, constraints):
#         select the random users and their behaviour with random latency
        self.noisy_tariff["h1_start"] = [random.choice(range(constraints["h1_start"]-2, 
                                                             constraints["h1_start"]+int(generate.duration/2))) for _ in range(len(self.avail_users))]
        self.noisy_tariff["h1_end"] = [random.choice(range(constraints["h1_end"]-int(generate.duration/2), 
                                                           constraints["h1_end"]+2)) for _ in range(len(self.avail_users))]
    

    def tariff_policy(self, levels, constraints):
#         use variables from auto_noise_addition and input variables of this function to create a tariff policy 
#         for each participating user **Needs more attention
        self.auto_noise_addition(levels,constraints)
    
        self.d = np.ones((48, len(self.avail_users)))
        self.df_tariff = pd.DataFrame(data=self.d, columns = self.avail_users)
        for i in range(len(self.avail_users)):
            self.df_tariff.loc[self.noisy_tariff["h1_start"][i]:self.noisy_tariff["h1_end"][i], self.avail_users[i]] = 2

        self.df_tariff.index = self.sample.index
        
        
    def run(self):
#         FOR EACH USER, call test function of consumption model, get modified behaviour, return original data point and modified data point
        self.sample = self.sample.interpolate(method = 'linear', axis = 0).ffill().bfill()
        self.sample = self.sample.join(self.df_weather.loc[self.sample.index,:])
        df_response = pd.DataFrame()
        self.sample["hour"] = self.sample.index.hour
        self.sample["day"] = self.sample.index.day
        self.sample["month"] = self.sample.index.month
        
        list_ = [i for i in range(len(self.avail_users))]

        for i in list_:
            one_hot= pd.get_dummies(self.df_tariff[self.avail_users[i]])
            one_hot_renamed = one_hot.rename(index=str, columns={1.0:'normal', 2.0:'high', 3.0:'low'}) 
            self.sample = pd.concat([self.sample, one_hot_renamed], axis =1)
            self.sample["low"] = 0

            self.sample["trial_n"] = self.sample[self.avail_users[i]]
            
#             consumption_model.test(self.sample[self.params['X_variables']], one_hot_renamed)
            self.test()
#             df_response[self.avail_users[i]] = consumption_model.preds
            df_response[self.avail_users[i]] = self.preds
            self.sample = self.sample.drop(['low', 'normal', 'high', 'trial_n'], axis= 1)
            
        df_response['response']= df_response.mean(axis = 1)
        return df_response['response']
            
            
            
    def test(self):
        self.preds = self.sample['trial_n']
        self.preds.loc[self.sample['high']==1] = self.preds.loc[self.sample['high']==1]*0.9 #(1 - 9/(100*self.params['active_users']*self.params['avail_users']))
        
        

In [ ]:
# class Simulator:
    
#     def __init__(self, df, df_weather, params):
#         self.params = params
#         self.df = df
#         self.df_weather = df_weather
#         active_users = int(len(df.columns)*self.params["active_users"])   # get no. of active users from input percentage
#         self.active_users = random.sample(list(df.columns), active_users)
#         self.noisy_tariff = {}
#         self.spring = [3, 4, 5]
#         self.summer = [6, 7, 8]
#         self.autumn = [9, 10, 11]
#         self.winter = [1, 2, 12]


#     def select_day(self):
# #         Get user ids of participating users
#         self.fuzzy_participation()
        
# #         Select the season
#         if self.params["season"] == -1:
#             month = random.randrange(1,12)
#         elif self.params["season"] == 0:
#             month = random.choice(self.spring)
#         elif self.params["season"] == 1:
#             month = random.choice(self.summer)
#         elif self.params["season"] == 2:
#             month = random.choice(self.autumn)
#         elif self.params["season"] == 3:
#             month = random.choice(self.winter)
            
# #         Select the day of week
#         if self.params["day_of_week"] == -1:
# #             Select random day
#             dow = random.randrange(0,7)
#         else:
#             dow = self.params["day_of_week"] 
            
# #         Select the random day from the entries which satisfy above conditions
#         shortlist = self.df.loc[(self.df.index.month == month) & (self.df.index.dayofweek == dow), :].index
#         day = random.choice(shortlist.day.values)
#         year = random.choice(shortlist.year.values)
#         timestamp = str(year)+"-"+str(month)+"-"+str(day)
#         self.sample = self.df.loc[timestamp,self.avail_users]
        
        
        
#     def random_day(self):
# #         Get user ids of participating users
#         self.fuzzy_participation()
    
# #         Sample a random day timestamp
#         shortlist = self.df.sample(axis = 0).index
#         day = random.choice(shortlist.day.values)
#         month = random.choice(shortlist.month.values)
#         year = random.choice(shortlist.year.values)
#         timestamp = str(year)+"-"+str(month)+"-"+str(day)
#         self.sample = self.df.loc[timestamp,self.avail_users]
        
        
    
#     def fuzzy_participation(self):
#         avail_users = int(len(self.active_users)*self.params["avail_users"])
#         self.avail_users = random.sample(self.active_users, avail_users)
    
    
#     def auto_noise_addition(self, levels, constraints):
# #         select the random users and their behaviour with random latency
#         self.noisy_tariff["h1_start"] = [random.choice(range(constraints["h1_start"]-2, 
#                                                              constraints["h1_start"]+int(generate.duration/2))) for _ in range(len(self.avail_users))]
#         self.noisy_tariff["h1_end"] = [random.choice(range(constraints["h1_end"]-int(generate.duration/2), 
#                                                            constraints["h1_end"]+2)) for _ in range(len(self.avail_users))]
    

#     def tariff_policy(self, levels, constraints):
# #         use variables from auto_noise_addition and input variables of this function to create a tariff policy 
# #         for each participating user **Needs more attention
#         self.auto_noise_addition(levels,constraints)
    
#         self.d = np.ones((48, len(self.avail_users)))
#         self.df_tariff = pd.DataFrame(data=self.d, columns = self.avail_users)
#         for i in range(len(self.avail_users)):
#             self.df_tariff.loc[self.noisy_tariff["h1_start"][i]:self.noisy_tariff["h1_end"][i], self.avail_users[i]] = 2

#         self.df_tariff.index = self.sample.index
        
        
#     def run(self):
# #         FOR EACH USER, call test function of consumption model, get modified behaviour, return original data point and modified data point
#         self.sample = self.sample.interpolate(method = 'linear', axis = 0).ffill().bfill()
#         self.sample = self.sample.join(self.df_weather.loc[self.sample.index,:])
#         self.df_response = pd.DataFrame()
#         response = pd.DataFrame()
#         self.sample.loc[:,"hour"] = self.sample.index.hour
#         self.sample.loc[:,"day"] = self.sample.index.day
#         self.sample.loc[:,"month"] = self.sample.index.month
        
#         list_ = [i for i in range(len(self.avail_users))]

#         with Pool(processes = 4, maxtasksperchild = 1000) as p:
#             p.map(self.loop, list_)
            
#         response.loc[:,'mean']= self.df_response.mean(axis = 1)
#         response.index = self.df_response.index
#         return response['mean']
            
#     def loop(self, i):
#         one_hot= pd.get_dummies(self.df_tariff[self.avail_users[i]])
#         one_hot_renamed = one_hot.rename(index=str, columns={1.0:'normal', 2.0:'high', 3.0:'low'}) 
#         self.sample = pd.concat([self.sample, one_hot_renamed], axis =1)
#         self.sample.loc[:,"low"] = 0

#         self.sample.loc[:,"trial_n"] = self.sample.loc[:,self.avail_users[i]]

# #             consumption_model.test(self.sample[self.params['X_variables']], one_hot_renamed)
#         self.test()
# #             df_response[self.avail_users[i]] = consumption_model.preds
#         self.df_response.loc[:,self.avail_users[i]] = self.preds
#         self.sample = self.sample.drop(['low', 'normal', 'high', 'trial_n'], axis= 1)
            
#     def test(self):
#         self.preds = self.sample.loc[:,'trial_n']
#         self.preds.loc[self.sample.loc[:,'high']==1] = self.preds.loc[self.sample.loc[:,'high']==1]*0.9 #(1 - 9/(100*self.params['active_users']*self.params['avail_users']))
        
        

In [ ]:
## calculation of percentage of energy consumption change

1- 8/(100*params['active_users']*params['avail_users'])

In [ ]:
df=pd.read_csv('../mod_datasets/aggregate_consumption.csv', sep=',', header=0, index_col=0, parse_dates=['GMT'], low_memory=False)
df = df.drop_duplicates()
df_midas=pd.read_csv('../mod_datasets/midas_weather.csv', sep=',', header=0, index_col=0, parse_dates=['GMT'], low_memory=False)
df_midas_rs = df_midas.resample('30T').mean()
df_interpolated = df_midas_rs.interpolate(method='linear')
df_weather = df_interpolated.loc['2013-01':'2013-12',:]
df_final = pd.concat([df,df_weather], axis=1)

df_n=pd.read_csv('../UKDA-7857-csv/csv/data_collection/data_tables/consumption_n.csv', sep=',', header=0, index_col=0, parse_dates=['GMT'], low_memory=False)
df_n = df_n.drop_duplicates()
df_weath = df_interpolated.copy()

In [ ]:
def import_data():
    print("Reading aggregate consumption data...")
    df=pd.read_csv('~/Documents/work/Active-Learning-TUD-Thesis/mod_datasets/aggregate_consumption.csv', sep=',', header=0, index_col=0, parse_dates=['GMT'], low_memory=False)
    df = df.drop_duplicates()
    print("Done")
    print("Reading weather data...")
    df_midas=pd.read_csv('~/Documents/work/Active-Learning-TUD-Thesis/mod_datasets/midas_weather.csv', sep=',', header=0, index_col=0, parse_dates=['GMT'], low_memory=False)
    df_midas_rs = df_midas.resample('30T').mean()
    df_interpolated = df_midas_rs.interpolate(method='linear')
    df_weather = df_interpolated.loc['2013-01':'2013-12',:]
    df_final = pd.concat([df,df_weather], axis=1)
    print("Done")
    print("Reading LCL consumption data...")
    df_n=pd.read_csv('~/Documents/work/Active-Learning-TUD-Thesis/UKDA-7857-csv/csv/data_collection/data_tables/consumption_n.csv', sep=',', header=0, index_col=0, parse_dates=['GMT'], low_memory=False)
    df_n = df_n.drop_duplicates()
    df_weath = df_interpolated.copy()
    print("Done")
    return df_final

In [ ]:
# Simulating dataframe time index for tariff policy
index = datetime(2013, 4, 10, 17, 30, 0)
h1_start = int(index.hour * 2) + int(index.minute / 30)
h1_end = h1_start + 10

constraints = {"h1_start": h1_start, "h1_end": h1_end}
level = 0       #dummy


sim = Simulator(df_n.loc['2012-05':, :], df_weath.loc['2012-05':, :], params)
sim.select_day()
# sim.sample


sim.tariff_policy(level, constraints)

response = sim.run()
df_response = pd.DataFrame(data=response, index = sim.sample.index)


output_notebook()
#     output_file("./temp/line.html") #Uncomment it to save the plot in html file
p=figure(x_axis_type = 'datetime',plot_width=800, plot_height=400)
p.line(sim.sample.index, response.values, line_width=1, color='blue')
p.line(sim.sample.index, sim.sample[sim.avail_users].mean(axis = 1).values, line_width=1, color='red')
show(p)



In [19]:
class activeLearner(object):
    
    def __init__(self, df_n, df_weath, params):
        self.df_n = df_n
        self.df_weath = df_weath
        self.params = params
        self.model = RandomForestClassifier()
        self.df = pd.DataFrame()
        
    def run(self):
        
        generate.experiment()
        # Add contextual data in future for the particular day to self.df
        level, constraints = generate.tariff()
        sim.tariff_policy(level, constraints)
        response = sim.run()
        self.df = pd.DataFrame(data=response, index = sim.sample.index)
        self.d = np.ones((48, 1))
        df_tariff = pd.DataFrame(data=self.d, columns = ["tariff"])
        df_tariff.loc[constraints["h1_start"]:constraints["h1_end"], "tariff"] = 2
        df_tariff.index = sim.sample.index
        self.df["tariff"] = df_tariff.tariff
        self.df["expected"] = sim.sample[sim.avail_users].mean(axis = 1).values

            
#        Create n number of datapoints from simulator (n=self.params["init_samples"])
#        Create a list of 1 to n for including a progress bar
        
        list_ = [i for i in range(self.params["init_samples"])]

        for i in tqdm(list_):
            
            generate.experiment()
            # Add contextual data in future for the particular day to temp_df
            level, constraints = generate.tariff()
            sim.tariff_policy(level, constraints)
            response = sim.run()
            temp_df = pd.DataFrame(data=response, index = sim.sample.index)
            #get tariff policy for this iteration
            df_tariff = pd.DataFrame(data=self.d, columns = ["tariff"])
            df_tariff.loc[constraints["h1_start"]:constraints["h1_end"], "tariff"] = 2
            df_tariff.index = sim.sample.index
            temp_df["tariff"] = df_tariff.tariff
            temp_df["expected"] = sim.sample[sim.avail_users].mean(axis = 1).values

            self.df = pd.concat([self.df, temp_df], axis=0, sort=True)
            
        
#         Create for loop to train the model for m number of times (where, m = self.params["total_experiments"] - self.params["init_samples"])
    
     
    
        
    def split_data(self, train_size):
        pass
        
        
        
    def train(self, class_weight):
        self.classifier = RandomForestClassifier(n_estimators=500, class_weight=class_weight)
        self.classifier.fit(self.X_train, self.y_train)
        self.test_y_predicted = self.classifier.predict(self.X_test)
        self.val_y_predicted = self.classifier.predict(self.X_val)
        return (self.X_train, self.X_val, self.X_test, self.val_y_predicted, self.test_y_predicted)
        

In [12]:
class randomGenerate(object):
    def __init__(self, params):
        self.params = params
        self.df = pd.DataFrame()
    
    def experiment(self):
        sim.random_day()
        self.df = sim.sample
        
        
    def tariff(self):
        self.year = random.randrange(2012,2013)
        self.month = random.randrange(1,12)
        self.day = random.randrange(1,28)
        self.hour = random.randrange(17,18)
        self.minute = random.choice([0,30])
        self.duration = random.randrange(6, 8)
        index = datetime(self.year, self.month, self.day, self.hour, self.minute, 0)
        h1_start = int(index.hour * 2) + int(index.minute / 30)
        h1_end = h1_start + self.duration
        constraints = {"h1_start": h1_start, "h1_end": h1_end}
        level = 0       #dummy
        return level, constraints

        
        
        
        

In [20]:
sim = Simulator(df_n.loc['2012-05':, :], df_weath.loc['2012-05':, :], params)
generate =  randomGenerate(params)
trials_ = activeLearner(df_n.loc['2012-05':, :], df_weath.loc['2012-05':, :], params)

In [21]:
trials_.run()
# trials_.df
# generate.experiment()
# generate.df

In [22]:
trials_.df

,mean,tariff,expected
GMT,,,
2012-06-03 00:00:00,0.131364,1.0,0.131364
2012-06-03 00:30:00,0.122168,1.0,0.122168
2012-06-03 01:00:00,0.110426,1.0,0.110426
2012-06-03 01:30:00,0.102160,1.0,0.102160
2012-06-03 02:00:00,0.098038,1.0,0.098038
2012-06-03 02:30:00,0.094645,1.0,0.094645
2012-06-03 03:00:00,0.092419,1.0,0.092419
2012-06-03 03:30:00,0.092168,1.0,0.092168
2012-06-03 04:00:00,0.087782,1.0,0.087782


In [ ]:
output_notebook()
#     output_file("./temp/line.html") #Uncomment it to save the plot in html file
p=figure(x_axis_type = 'datetime',plot_width=800, plot_height=400)
p.line(sim.sample.index, sim.sample[sim.avail_users].mean(axis = 1).values, line_width=1, color='red')
p.line(sim.sample.index, trials_.df["mean"].values, line_width=1, color='blue')

show(p)

In [ ]:
sample_size = [i for i in range(params["init_samples"])]
sample_size

In [ ]:
# # sim.sample = 0

# output_notebook()
# #     output_file("./temp/line.html") #Uncomment it to save the plot in html file
# p=figure(x_axis_type = 'datetime',plot_width=800, plot_height=400)
# p.line(sim.sample.index, response.values, line_width=1, color='blue')
# p.line(sim.sample.index, sim.sample[sim.avail_users].mean(axis = 1).values, line_width=1, color='red')
# show(p)



# # response
# # sim.sample[sim.avail_users].mean(axis = 1)

In [ ]:
corr = df_final.corr()
sns.heatmap(corr,
            xticklabels=corr.columns,
            yticklabels=corr.columns)